# 1. Ambil Data Dari Gdelt, Ambil yang Nopember Saja

In [1]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'

# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# separate out those links that begin with four digits 
file_list = [x for x in link_list if (str(x[0:6]) == '201811')]
print(file_list)
#print("Done!")

['20181103.export.CSV.zip', '20181102.export.CSV.zip', '20181101.export.CSV.zip']


# 2. Masukkan Data ke Dalam Format TSV untuk Data Dari Negara Indonesia

In [2]:
infilecounter = 0
outfilecounter = 0

import os.path
import urllib
import zipfile
import glob
import operator

local_path = 'D:/GDELT/'
dir_country = 'country'
fips_country_code = 'ID'

for compressed_file in file_list[infilecounter:]:
    print(compressed_file),
    
    # if we dont have the compressed file stored locally, go get it. Keep trying if necessary.
    while not os.path.isfile(local_path+compressed_file): 
        print('Downloading....')
        urllib.request.urlretrieve(url=gdelt_base_url + compressed_file,
                           filename=local_path + compressed_file)
    
    # extract the contents of the compressed file to a temporary directory    
    print('extracting,')
    z = zipfile.ZipFile(file=local_path+compressed_file, mode='r')    
    z.extractall(path=local_path+'tmp/')
    
    # parse each of the csv files in the working directory, 
    print('parsing,')
    
    for infile_name in glob.glob(local_path+'tmp/*'):
        outfile_name = local_path+dir_country+'/'+fips_country_code+'%04i.tsv'%outfilecounter
        
        try:
            # Create target Directory
            os.mkdir(local_path+dir_country)
            print("Directory " , dir_country ,  " Created ") 
        except FileExistsError:
            print("Directory " , dir_country ,  " already exists")
        
        # open the infile and outfile
        print(infile_name)
        with open(infile_name,mode='r') as infile, open(outfile_name,mode='w') as outfile:
            try:
                for line in infile:
                    # extract lines with our interest country code
                    if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):    
                        outfile.write(line)
            except:
                print("")
            outfilecounter +=1
            
        # delete the temporary file
        os.remove(infile_name)
    infilecounter +=1
    print('done')

20181103.export.CSV.zip
extracting,
parsing,
Directory  country  already exists
D:/GDELT/tmp\20181103.export.CSV

done
20181102.export.CSV.zip
extracting,
parsing,
Directory  country  already exists
D:/GDELT/tmp\20181102.export.CSV

done
20181101.export.CSV.zip
extracting,
parsing,
Directory  country  already exists
D:/GDELT/tmp\20181101.export.CSV

done


# 3. Masukkan Data Dari TSV ke Database

In [3]:
import glob
import pymysql
import pandas as pd

host = '127.0.0.1'
port = "3306"
user = 'root'
password = ''
database = 'news'

conn = pymysql.connect(
    host=host,
    port=int(port),
    user=user,
    passwd=password,
    db=database,
    charset='utf8mb4')

def run(sql):
    df = pd.read_sql_query(sql,conn)
    return df
x = conn.cursor()

# Build DataFrames from each of the intermediary files
files = glob.glob(local_path+'country/'+fips_country_code+'*')
DFlist = []
print("We are working on it, please take your coffee...")  
for active_file in files:
    with open(active_file,mode='r') as xfile:
        for line in xfile:
            kolom = line.split("\t")
            kolomID = kolom[0]
            kolomURL = kolom[57]
       
            sql = "INSERT INTO gdelt_content (GLOBALEVENTID,URL) SELECT * FROM (SELECT '"+kolomID+"','"+kolomURL+"') AS tmp WHERE NOT EXISTS (SELECT GLOBALEVENTID FROM gdelt_content WHERE GLOBALEVENTID = '"+kolomID+"') LIMIT 1"
            #print(sql)
            x.execute(sql)
            conn.commit()
print("Done! All have finished")
    
    
    



We are working on it, please take your coffee...
Done! All have finished


# 4. Update Title dan Content data di Database 

In [4]:
from newspaper import Article
import pymysql.cursors
import sys

offset = sys.argv[1]
limit = sys.argv[2]
# year = sys.argv[3]

connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='news',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

hdr = {'User-Agent': 'Mozilla/5.0'}
print('Take your coffee again! We will do this job for you...')
with connection.cursor() as cursor:
    sql = "SELECT * FROM `gdelt_content` WHERE (TITLE IS NULL)"
    cursor.execute(sql)
    print(sql)
    result = cursor.fetchall()
    for r in result:
        try:
            r["URL"] = r["URL"].split('<UDIV>')
            print(r["URL"][0])
            article = Article(r["URL"][0])
            article.download()
            article.parse()
            with connection.cursor() as cursor:
                sql = "UPDATE `gdelt_content` SET title=%s,content=%s WHERE GLOBALEVENTID=%s"
                cursor.execute(sql, (article.title.encode('utf-8'), article.text.encode('utf-8'),r["GLOBALEVENTID"]))
                connection.commit()
        except:
            pass

print("Done!")

Take your coffee again! We will do this job for you...
SELECT * FROM `gdelt_content` WHERE (TITLE IS NULL)
https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=12152702

https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=12152702

https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=12152702

https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=12152702

https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=12152702

https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=12152702

https://www.sbs.com.au/news/no-more-global-missions-for-malcolm-turnbull

https://www.theage.com.au/world/asia/divers-zero-in-on-lion-air-flight-610-black-boxes-20181101-p50db8.html

https://www.theage.com.au/world/asia/divers-zero-in-on-lion-air-flight-610-black-boxes-20181101-p50db8.html

https://www.sbs.com.au/news/no-more-global-missions-for-malcolm-turnbull

https://www.theage.com.au/world/asia/divers-zero-in-on-lion-air-flight-

http://www.santafenewmexican.com/news/indonesian-divers-hear-pings-as-they-zero-in-on-lion/article_fc8ec4b8-f627-50fd-a54f-ebdbcd70d6bc.html

https://www.msn.com/en-gb/news/other/turnbull-contradicts-morrison-over-indonesia-trip-after-alan-jones-interview/ar-BBPbX01

https://www.msn.com/en-au/news/other/turnbull-contradicts-morrison-over-indonesia-trip-after-alan-jones-interview/ar-BBPbZKh

https://www.msn.com/en-gb/news/other/turnbull-contradicts-morrison-over-indonesia-trip-after-alan-jones-interview/ar-BBPbX01

https://www.msn.com/en-au/news/other/turnbull-contradicts-morrison-over-indonesia-trip-after-alan-jones-interview/ar-BBPbZKh

https://www.msn.com/en-gb/news/other/turnbull-contradicts-morrison-over-indonesia-trip-after-alan-jones-interview/ar-BBPbX01

https://www.msn.com/en-au/news/other/turnbull-contradicts-morrison-over-indonesia-trip-after-alan-jones-interview/ar-BBPbZKh

https://www.msn.com/en-gb/news/other/turnbull-contradicts-morrison-over-indonesia-trip-after-alan-jone

http://www.thejakartapost.com/news/2018/11/01/cheap-tickets-multiple-destinations-direct-flight-make-lion-air-favorite.html

http://www.thejakartapost.com/news/2018/11/01/cheap-tickets-multiple-destinations-direct-flight-make-lion-air-favorite.html

http://www.thejakartapost.com/news/2018/11/01/cheap-tickets-multiple-destinations-direct-flight-make-lion-air-favorite.html

http://www.thejakartapost.com/news/2018/11/01/cheap-tickets-multiple-destinations-direct-flight-make-lion-air-favorite.html

http://www.thejakartapost.com/news/2018/11/01/cheap-tickets-multiple-destinations-direct-flight-make-lion-air-favorite.html

http://www.thejakartapost.com/news/2018/11/01/cheap-tickets-multiple-destinations-direct-flight-make-lion-air-favorite.html

http://www.thejakartapost.com/news/2018/11/01/cheap-tickets-multiple-destinations-direct-flight-make-lion-air-favorite.html

http://www.thejakartapost.com/news/2018/11/01/cheap-tickets-multiple-destinations-direct-flight-make-lion-air-favorite.html



http://www.msn.com/en-us/news/world/indonesia-divers-retrieve-black-box-from-crashed-jet-media/ar-BBPc7hg

https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=12152882

http://www.msn.com/en-us/news/world/indonesia-divers-retrieve-black-box-from-crashed-jet-media/ar-BBPc7hg

https://www.japantimes.co.jp/news/2018/11/01/asia-pacific/moon-jae-says-north-korean-leader-kim-jong-un-will-visit-seoul-soon/

https://www.japantimes.co.jp/news/2018/11/01/asia-pacific/moon-jae-says-north-korean-leader-kim-jong-un-will-visit-seoul-soon/

https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=12152882

https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=12152882

https://www.manilatimes.net/skorean-court-allows-conscientious-objection-to-military-service/460630/

https://www.japantimes.co.jp/news/2018/11/01/asia-pacific/moon-jae-says-north-korean-leader-kim-jong-un-will-visit-seoul-soon/

https://www.japantimes.co.jp/news/2018/11/01/asia-pacific/moon-jae-says-

https://www.malaysiakini.com/c/en/news/233a034f4a9f6da2f9df8ba880a023f6

https://www.malaysiakini.com/c/en/news/233a034f4a9f6da2f9df8ba880a023f6

https://www.malaysiakini.com/c/en/news/233a034f4a9f6da2f9df8ba880a023f6

https://www.upstreamonline.com/hardcopy/1619084/inpex-urged-to-file-plan-for-abadi-this-month

https://www.malaysiakini.com/c/en/news/233a034f4a9f6da2f9df8ba880a023f6

https://www.upstreamonline.com/hardcopy/1619084/inpex-urged-to-file-plan-for-abadi-this-month

https://www.malaysiakini.com/c/en/news/233a034f4a9f6da2f9df8ba880a023f6

https://www.upstreamonline.com/hardcopy/1619084/inpex-urged-to-file-plan-for-abadi-this-month

https://www.upstreamonline.com/hardcopy/1619084/inpex-urged-to-file-plan-for-abadi-this-month

https://www.upstreamonline.com/hardcopy/1619084/inpex-urged-to-file-plan-for-abadi-this-month

https://www.malaysiakini.com/c/en/news/233a034f4a9f6da2f9df8ba880a023f6

https://www.malaysiakini.com/c/en/news/233a034f4a9f6da2f9df8ba880a023f6

https://www.up

...

https://www.independent.co.uk/arts-entertainment/films/news/eminem-bodied-free-cinema-producer-2018-calum-worthy-rap-a8614756.html

https://www.independent.co.uk/arts-entertainment/films/news/eminem-bodied-free-cinema-producer-2018-calum-worthy-rap-a8614756.html

https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=12153967

https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=12153967

https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=12153967

https://www.yahoo.com/news/details-highlight-lion-air-jets-problems-crash-050911889.html

https://www.nzherald.co.nz/world/news/article.cfm?c_id=2&objectid=12153967

https://www.yahoo.com/news/details-highlight-lion-air-jets-problems-crash-050911889.html

https://www.independent.co.uk/arts-entertainment/films/news/eminem-bodied-free-cinema-producer-2018-calum-worthy-rap-a8614756.html

https://www.independent.co.uk/arts-entertainment/tv/news/will-and-grace-me-too-debra-messing-robert-klein-eric-mccormack

http://www.newsdzezimbabwe.co.uk/2018/11/ncube-wants-mangudya-out.html

http://www.newsdzezimbabwe.co.uk/2018/11/ncube-wants-mangudya-out.html

http://www.newsdzezimbabwe.co.uk/2018/11/ncube-wants-mangudya-out.html

http://www.newsdzezimbabwe.co.uk/2018/11/ncube-wants-mangudya-out.html

http://www.newsdzezimbabwe.co.uk/2018/11/ncube-wants-mangudya-out.html

http://www.newsdzezimbabwe.co.uk/2018/11/ncube-wants-mangudya-out.html

https://timesofindia.indiatimes.com/city/surat/gjepc-seeks-centres-help-for-ailing-diamond-sector/articleshow/66481781.cms

https://www.firstpost.com/world/indonesian-plane-crashes-into-sea-all-189-on-board-feared-dead-5496391.html

https://www.firstpost.com/world/indonesian-plane-crashes-into-sea-all-189-on-board-feared-dead-5496391.html

https://www.firstpost.com/world/indonesian-plane-crashes-into-sea-all-189-on-board-feared-dead-5496391.html

https://www.firstpost.com/world/indonesian-plane-crashes-into-sea-all-189-on-board-feared-dead-5496391.html

https://

https://www.theguardian.com/world/2018/nov/03/why-did-it-fly-grief-mixes-with-anger-over-crashed-lion-air-flight

https://jakartaglobe.id/news/divers-to-check-if-ping-signals-reveal-location-of-crashed-lion-air-plane/

https://jakartaglobe.id/news/divers-to-check-if-ping-signals-reveal-location-of-crashed-lion-air-plane/

https://jakartaglobe.id/news/divers-to-check-if-ping-signals-reveal-location-of-crashed-lion-air-plane/

https://jakartaglobe.id/news/divers-to-check-if-ping-signals-reveal-location-of-crashed-lion-air-plane/

https://jakartaglobe.id/disasters/rescue-workers-believe-they-have-located-crashed-lion-air-jets-fuselage-and-black-box/

https://jakartaglobe.id/news/divers-to-check-if-ping-signals-reveal-location-of-crashed-lion-air-plane/

https://jakartaglobe.id/news/divers-to-check-if-ping-signals-reveal-location-of-crashed-lion-air-plane/

http://www.xinhuanet.com/english/2018-11/03/c_137578600.htm

https://www.theguardian.com/world/2018/nov/03/why-did-it-fly-grief-mixes-

https://www.freemalaysiatoday.com/category/world/2018/11/03/diver-dies-in-search-for-indonesia-jet-crash-dead/

https://www.rappler.com/world/regions/south-central-asia/215836-diver-dies-search-indonesia-jet-crash-dead

https://www.jacarandafm.com/news/news/diver-dies-search-indonesia-jet-crash-dead/

https://www.rappler.com/world/regions/south-central-asia/215836-diver-dies-search-indonesia-jet-crash-dead

https://www.rappler.com/world/regions/south-central-asia/215836-diver-dies-search-indonesia-jet-crash-dead

http://www.saudigazette.com.sa/article/547022/World/Asia/Indonesia-struggles-with-damaged-black-box-from-crashed-jet-hunts-for-second

https://www.rappler.com/world/regions/south-central-asia/215836-diver-dies-search-indonesia-jet-crash-dead

https://www.rappler.com/world/regions/south-central-asia/215836-diver-dies-search-indonesia-jet-crash-dead

https://www.rappler.com/world/regions/south-central-asia/215836-diver-dies-search-indonesia-jet-crash-dead

https://www.rappler.co

C:\Users\Anaconda\lib\site-packages\pymysql\cursors.py:329: Warning: (1366, "Incorrect string value: '\\xC4\\x93z an...' for column 'content' at row 1")
  self._do_get_result()
